In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
tf.set_random_seed(777) # reproducibility

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [6]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [7]:
class Model:
  def __init__(self, sess, name):
    self.sess = sess
    self.name = name
    self._build_net()
  
  def _build_net(self):
    with tf.variable_scope(self.name):
      # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
      self.training = tf.placeholder(tf.bool)

      # input place holders
      self.X = tf.placeholder(tf.float32, [None, 784])

      # img 28x28x1 (black/white), Input Layer
      X_img = tf.reshape(self.X, [-1, 28, 28, 1])
      self.Y = tf.placeholder(tf.float32, [None, 10])

      # Convolutional Layer #1
      conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3, 3],
                               padding = "SAME", activation = tf.nn.relu)
      # Pooling Layer #1
      pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2],
                                      padding = "SAME", strides = 2)
      dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.3, training = self.training)

      # Convolutional Layer #2 and Pooling Layer #2
      conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size = [3, 3],
                               padding = "SAME", activation = tf.nn.relu)
      pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = [2, 2],
                                      padding = "SAME", strides = 2)
      dropout2 = tf.layers.dropout(inputs = pool2, rate = 0.3, training = self.training)

      # Convolutional Layer #3 and Pooling Layer #3
      conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size = [3, 3],
                               padding = "same", activation = tf.nn.relu)
      pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2, 2],
                                      padding = "same", strides = 2)
      dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.3, training = self.training)

      # Dense Layer with Relu
      flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
      dense4 = tf.layers.dense(inputs = flat, units = 625, activation = tf.nn.relu)
      dropout4 = tf.layers.dropout(inputs = dense4, rate = 0.5, training = self.training)

      # Logits (no activation) Layer: L5 Final FC 625 inputs -> 10 outputs
      self.logits = tf.layers.dense(inputs = dropout4, units = 10)
    
    # define cost/loss & optimizer
    self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits = self.logits, labels = self.Y
    ))
    self.optimizer = tf.train.AdamOptimizer(
        learning_rate = learning_rate
    ).minimize(self.cost)

    correct_prediction = tf.equal(
        tf.argmax(self.logits, 1), tf.argmax(self.Y, 1)
    )
    self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  
  def predict(self, x_test, training = False):
    return self.sess.run(self.logits,
                         feed_dict = {self.X: x_test, self.training: training})
  
  def get_accuracy(self, x_test, y_test, training = False):
    return self.sess.run(self.accuracy,
                         feed_dict = {self.X: x_test, self.Y: y_test, self.training: training})
  
  def train(self, x_data, y_data, training = True):
    return self.sess.run([self.cost, self.optimizer],
                         feed_dict = {self.X: x_data, self.Y: y_data, self.training: training})

In [8]:
# initialize
sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
print('Learning Started!')

# train my model
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = int(mnist.train.num_examples / batch_size)

  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    c, _ = m1.train(batch_xs, batch_ys)
    avg_cost += c / total_batch
  
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

Learning Started!
Epoch: 0001 cost = 0.283332967
Epoch: 0002 cost = 0.085422711
Epoch: 0003 cost = 0.067209122
Epoch: 0004 cost = 0.055389582
Epoch: 0005 cost = 0.048674222
Epoch: 0006 cost = 0.044418247
Epoch: 0007 cost = 0.040088436
Epoch: 0008 cost = 0.038217998
Epoch: 0009 cost = 0.035545900
Epoch: 0010 cost = 0.032474376
Epoch: 0011 cost = 0.031684107
Epoch: 0012 cost = 0.030804179
Epoch: 0013 cost = 0.029891058
Epoch: 0014 cost = 0.027687727
Epoch: 0015 cost = 0.025958249
Learning Finished!


In [10]:
# Test model and check accuracy
print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.labels))

Accuracy: 0.9927
